# Smart Data Model Translation

The purpose of this Jupyter Notebook is to translate NGSI-LD compatible Smart Data Models represented as JSON-Schema into an OWL ontology. The Smart Data Models are available from https://github.com/smart-data-models.


The following code pieces
* import libraries
* initialize global variables
* define constants

In [1]:
import json
import sys
includes = {}
definitions = {}
namespaces = {}
reverse_namespeace = {}
properties = []


In [2]:
# TODO correct handling of array
ALL_OF = "allOf"
ANY_OF = "anyOf"
ARRAY = "array"
AT_ID = "@id"
AT_TYPE = "@type"
DATE_TIME = "date-time"
DEFINITIONS = "definitions"
DESCRIPTION = "description"
DOLLAR_ID = "$id"
DOLLAR_REF = "$ref"
ENTITY_IDENTIFIER_TYPE = "EntityIdentifierType"
ENTITY_IDENTIFIER_URI = "https://smart-data-models.github.io/data-models/common-schema.json#/definitions/EntityIdentifierType"
ENTITY_URI = "https://uri.etsi.org/ngsi-ld/Entity"
ENUM = "enum"
FORMAT = "format"
HEADING = "heading"
ID = "id"
ID_FORMAT = "'format': 'uri'"
ITEMS = "items"
ITEM_TYPE = "item_type"
ITEM_ENUM = "item_enum"
ITEM_FORMAT = "item_format"
JSON_SIMPLE_TYPES = ["string", "number", "integer", "date-time", "array", "boolean"]
LOCATION = "location"
LOCATION_URI = "https://smart-data-models.github.io/data-models/common-schema.json#/definitions/Location-Commons/properties/location"
MIN_CARDINALITY_1 = "\"1\"^^xsd:nonNegativeInteger"
NGSILD_ENTITY = "https://uri.etsi.org/ngsi-ld/Entity"
NGSILD_GEOPROPERTY = "https://uri.etsi.org/ngsi-ld/GeoProperty"
NGSILD_LANGUAGE_PROPERTY = "https://uri.etsi.org/ngsi-ld/LanguageProperty"
NGSILD_PROPERTY = "https://uri.etsi.org/ngsi-ld/Property"
NGSILD_RELATIONSHIP = "https://uri.etsi.org/ngsi-ld/Relationship"
NGSILD_TYPE = "ngsild_type"
NGSILD_VALUE = "https://uri.etsi.org/ngsi-ld/Value"
NUMBER = "number"
OBJECT = "object"
ONE_OF = "oneOf"
OWL_CLASS = "owl:Class"
OWL_DATA_UNION_OF = "owl:DataUnionOf"
OWL_DATATYPE_PROPERTY = "owl:DatatypeProperty"
OWL_DISJOINT_WITH = "owl:disjointWith"
OWL_INTERSECTION_OF = "owl:intersectionOf"
OWL_MIN_CARDINALITY_Q = "owl:minCardinalityQ"
OWL_OBJECT_PROPERTY = "owl:ObjectProperty"
OWL_ON_DATATYPE = "owl:onDatatype"
OWL_ONE_OF = "owl:oneOf"
OWL_ON_PROPERTY = "owl:onProperty"
OWL_PROPERTY = "owl_property"
OWL_RESTRICTION = "owl:Restriction"
OWL_UNION_OF = "owl:unionOf"
OWL_WITH_RESTRICTIONS = "owl:withRestrictions"
PROPERTIES = "properties"
RDF_FIRST = "rdf:first"
RDF_LIST = "rdf:List"
RDF_NIL = "rdf:nil"
RDF_REST = "rdf:rest"
RDF_TYPE = "rdf:type"
RDFS_COMMENT = "rdfs:comment"
RDFS_DATATYPE = "rdfs:Datatype"
RDFS_DOMAIN = "rdfs:domain"
RDFS_SUBCLASS_OF = "rdfs:subClassOf"
RDFS_SUBPROPERTY_OF = "rdfs:subPropertyOf"
RDFS_RANGE = "rdfs:range"
RELATIONSHIP = "Relationship"
REQUIRED = "required"
RESTRICTIONS = ["xsd:pattern", "xsd:minLength", "xsd:maxLength", "rdf:langRange", "xsd:maxInclusive", " xsd:minInclusive"]
SEE_ALSO = "seeAlso"
SPEED = "speed"
STRING = "string"
TITLE = "title"
TYPE = "type"
TYPE_POSTFIX = "Type"
TYPE_TRANSLATION = {
    "string": "xsd:string",
    "integer": "xsd:integer",
    "date-time": "xsd:dateTime",
    "boolean": "xsd:boolean",
    "number": "xsd:float",
    "pattern": "xsd:pattern",
    "minLength": "xsd:minLength",
    "maxLength": "xsd:maxLength",
    "description": "rdfs:comment",
    "uri": "xsd:anyURI",
    "array": "rdf:List"
}
URI = "uri"
URI_TRANSLATION = {
    "https://uri.etsi.org/ngsi-ld/default-context/timeInstant": "https://smart-data-models.github.io/data-models/common-schema.json#/definitions/timeInstant",
    "https://uri.etsi.org/ngsi-ld/default-context/userAlias": "https://smart-data-models.github.io/data-models/common-schema.json#/definitions/userAlias",
    "https://uri.etsi.org/ngsi-ld/default-context/dateYearLess": "https://smart-data-models.github.io/data-models/common-schema.json#/definitions/dateYearLess"
}
VALUE = "value"
VALUES = "values"
XSD_DATETIME= "xsd:dateTime"
XSD_PATTERN = "xsd:pattern"
XSD_ANYURI = "xsd:anyURI"
XSD_STRING = "xsd:string"




### Helper functionality
* read file from URL
* read OffStreetParking as example --> data (currently used by main function)
* remove prefix

In [3]:
import urllib.request

def readFromURL(url):
    response = urllib.request.urlopen(url)
    data = response.read()      # a `bytes` object
    text = data.decode('utf-8') # a `str`; this step can't be used if data is binary
    data = json.loads(text)
    return data


In [4]:
#data = readFromURL("https://raw.githubusercontent.com/smart-data-models/dataModel.Parking/master/OffStreetParking/schema.json")
data = readFromURL("https://raw.githubusercontent.com/smart-data-models/dataModel.ParksAndGardens/master/Garden/schema.json")
data

{'$schema': 'https://json-schema.org/draft/2020-12/schema',
 '$schemaVersion': '0.0.4',
 '$id': 'https://smart-data-models.github.io/dataModel/ParksAndGardens/Garden/schema.json',
 'title': 'Smart Data Models - Parks and Gardens / Garden',
 'description': 'A garden is a distinguishable planned space, usually outdoors, set aside for the display, cultivation, and enjoyment of plants and other forms of nature.',
 'type': 'object',
 'allOf': [{'$ref': 'https://smart-data-models.github.io/data-models/common-schema.json#/definitions/GSMA-Commons'},
  {'$ref': 'https://smart-data-models.github.io/data-models/common-schema.json#/definitions/PhysicalObject-Commons'},
  {'properties': {'location': {'$ref': 'https://smart-data-models.github.io/data-models/common-schema.json#/definitions/Location-Commons/properties/location'},
    'address': {'$ref': 'https://smart-data-models.github.io/data-models/common-schema.json#/definitions/Location-Commons/properties/address'},
    'type': {'type': 'string'

In [5]:
def removeprefix(self: str, prefix: str, /) -> str:
    if self.startswith(prefix):
        return self[len(prefix):]
    else:
        return self[:]

In [6]:
def separateCoreURLfromName(url):
    splitResult = []
    if isinstance(url, dict):
        url = url['@id']
    if "terms.jsonld#/definitions/" in url:
        lastSlash = url.rfind("/")
        splitResult.append(url[:lastSlash +1])
        splitResult.append(url[lastSlash + 1:]) 
    elif "common-schema.json#/definitions/" in url:
        lastSlash = url.rfind("/")
        splitResult.append(url[:lastSlash +1])
        splitResult.append(url[lastSlash +1:])
    elif '#' in url:
        splitURL = url.split('#')
        splitResult.append(splitURL[0] + "#")
        splitResult.append(splitURL[1])
    elif '/' in url:
        lastSlash = url.rfind("/")
        splitResult.append(url[:lastSlash +1])
        splitResult.append(url[lastSlash + 1:])
    elif ':' in url:
        lastColon = url.rfind(":")
        splitResult.append(url[:lastColon +1])
        splitResult.append(url[lastColon + 1:])
    else:
        splitResult.append('')
        splitResult.append(url)
    return splitResult

result = separateCoreURLfromName("https://smart-data-models.github.io/data-models/terms.jsonld#/definitions/dateCreated")
print(result[0] + " - " + result[1]) 
result = separateCoreURLfromName("http://uri.etsi.org/ngsi-ld/location")
print(result[0] + " - " + result[1]) 

https://smart-data-models.github.io/data-models/terms.jsonld#/definitions/ - dateCreated
http://uri.etsi.org/ngsi-ld/ - location


In [7]:
def usePrefix(url):
    splitResult = separateCoreURLfromName(url)
    if splitResult[0] in inversePrefixes:
        return inversePrefixes[splitResult[0]]+splitResult[1]
    elif splitResult[0] in prefixes:
        return splitResult[0]+splitResult[1]        
    else:
        return url

#print(usePrefix("https://smart-data-models.github.io/data-models/terms.jsonld#/definitions/dateCreated"))
#print(usePrefix("https://uri.etsi.org/ngsi-ld/name"))
#print(usePrefix("https://uri.fiware.org/ns/data-models#seeAlso"))
#print(usePrefix("http://uri.etsi.org/ngsi-ld/location"))
#dictionary = {'@container': '@list', '@id': 'geojson:bbox'}
#print(usePrefix(dictionary))
#for element in context:
#    print(usePrefix(context[element]))

#prefixes['agrifood']="<https://smartdatamodels.org/dataModel.Agrifood>"
#inversePrefixes["https://smartdatamodels.org/dataModel.Agrifood/"] = "agrifood:"

#print(usePrefix("https://smartdatamodels.org/dataModel.Agrifood/additionalInfo"))

### Read @context
* Read Smart Data Model @context (currently incomplete!)
* Read NGSI-LD core @context

In [8]:
#smartDataModelContextURL = 'https://raw.githubusercontent.com/smart-data-models/dataModel.Parking/master/context.jsonld'
smartDataModelContextURL = 'https://smart-data-models.github.io/data-models/context.jsonld'
def readJSONLDContext(url):
    data = readFromURL(url)
    context = data['@context']
    return context

context = readJSONLDContext(smartDataModelContextURL)
print(context['OffStreetParking'])

https://uri.fiware.org/ns/data-models#OffStreetParking


In [9]:
coreContextURL = 'https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context-v1.8.jsonld'
coreContext = readJSONLDContext(coreContextURL)
uri=coreContext['ngsi-ld']
print(coreContext['ngsi-ld'])
prefix = coreContext['Property'].split(':')[0]
print('Prefix: ' + prefix)

print(uri+removeprefix(coreContext['Property'],prefix + ":"))


https://uri.etsi.org/ngsi-ld/
Prefix: ngsi-ld
https://uri.etsi.org/ngsi-ld/Property


### NGSI-LD Base Ontology
* read ontology from file

serves as basis for Smart Data Model ontology

In [10]:
prefixesRead = []
prefixes = {}
inversePrefixes = {}
classes = {}
objectProperties = {}
datatypeProperties = {}
ngsildClasses = []
ngsildObjectProperties = []
ngsildDatatypeProperties = []
baseInfo = []
defaultURL = "https://uri.etsi.org/ngsi-ld/default-context/"

def parsePrefixes():
    global prefixes
    global baseInfo
    global inversePrefixes
    prefixes = {}
    inversePrefixes = {}
    baseInfo = []
    for l in prefixesRead:
        if l.startswith("@prefix"):
            prefix = {}
            elements = l.split()
            key = elements[1]
            value = elements[2]
            prefixes[key] = value
            inverseKey = value[1:len(value)-1]
            inversePrefixes[inverseKey]=key
        else:
            if 'http' in l:
                baseInfo.append(l)
    prefixes["smart:"] = "<https://smartdatamodels.org/>"
    inversePrefixes["https://smartdatamodels.org/"] = "smart:"
    prefixes["fiware:"] = "<https://uri.fiware.org/ns/data-models#>"
    inversePrefixes["https://uri.fiware.org/ns/data-models#"] = "fiware:"
    prefixes["terms:"] ="<https://smart-data-models.github.io/data-models/terms.jsonld#/definitions/>"
    inversePrefixes["https://smart-data-models.github.io/data-models/terms.jsonld#/definitions/"] = "terms:"
    prefixes["definitions:"]="<https://smart-data-models.github.io/data-models/common-schema.json#/definitions/>"
    inversePrefixes["https://smart-data-models.github.io/data-models/common-schema.json#/definitions/"] = "definitions:"
    prefixes["common:"]="<https://smart-data-models.github.io/data-models/common-schema.json#/definitions/Location-Commons/properties/>"
    inversePrefixes["https://smart-data-models.github.io/data-models/common-schema.json#/definitions/Location-Commons/properties/"] = "common:"
    prefixes["saref:"] = "<https://w3id.org/def/saref#>"
    inversePrefixes["https://w3id.org/def/saref#"] = "saref:"
    prefixes["saref4agri:"] = "<https://w3id.org/def/saref4agri#>"
    inversePrefixes["https://w3id.org/def/saref4agri#"] = "saref4agri:"
    prefixes["goodrelations:"] = "<http://purl.org/goodrelations/v1#>"
    inversePrefixes["http://purl.org/goodrelations/v1#"] = "goodrelations:"
    prefixes["geojson:"] = "<https://purl.org/geojson/vocab#>"
    inversePrefixes["https://purl.org/geojson/vocab#"] = "geojson:"
    prefixes["purlterms:"] = "<http://purl.org/dc/terms/>"
    inversePrefixes["http://purl.org/dc/terms/"] = "purlterms:"
    prefixes["schema:"] = "<https://schema.org/>"
    inversePrefixes["https://schema.org/"] = "schema:"   
    prefixes["ngsi-ld:"] = "<http://uri.etsi.org/ngsi-ld/>"
    inversePrefixes["http://uri.etsi.org/ngsi-ld/"] = "ngsi-ld:"
    prefixes["default:"] = "<" + defaultURL + ">"
    inversePrefixes[defaultURL] = "default:"

def parseNGSILDClasses():
    global ngsildClasses
    global classes
    
    ngsiClass = {}
    classname = ""
    subelements=[]
    readSubclasses = False
    readDisjointWith = False
    for l in ngsildClasses:
        if(l.startswith("###")):
            if(not(classname=="")):
                classes[classname]=ngsiClass
                ngsiClass={}
            classname = l[5:len(l)-1]
            ngsiClass[RDF_TYPE]=OWL_CLASS
        if(l.find(RDFS_SUBCLASS_OF)>-1):
            elements = l.split()
            if(elements[2]==','):
                readSubclasses = True
                subelements=[]
                subelements.append(elements[1])
            else:
                ngsiClass[RDFS_SUBCLASS_OF]=elements[1]
        if readSubclasses:
            elements = l.split()
            if(len(elements)>0):
                if(elements[1]==','):
                    subelements.append(elements[0])
                elif(elements[1]=='.'):
                    subelements.append(elements[0])
                    ngsiClass[RDFS_SUBCLASS_OF]=subelements
                    readSubclasses = False     

        if(l.find(OWL_DISJOINT_WITH)>-1):
            elements = l.split()
            if(elements[2]==','):
                readDisjointWith = True
                subelements=[]
                subelements.append(elements[1])
            else:
                ngsiClass[OWL_DISJOINT_WITH]=elements[1]
        if readDisjointWith:
            elements = l.split()
            if(len(elements)>0):
                if(elements[1]==','):
                    subelements.append(elements[0])
                elif(elements[1]=='.'):
                    subelements.append(elements[0])
                    ngsiClass[OWL_DISJOINT_WITH]=subelements
                    readDisjointWith = False     

                
    if(not(classname=="")):
        classes[classname]=ngsiClass
    #for name in classes.keys():
    #   print(name + str(classes[name]))


def parseNGSILDObjectProperties():
    global ngsilObjectProperties
    global objectProperties
    
    ngsiObjectProperty = {}
    propertyName = ""
    subelements=[]
    rdftype = {}
    unionOf = {}
    readDomain = False
    readRange = False
    readUnionOf = False
    uri=coreContext['ngsi-ld']
    for l in ngsildObjectProperties:
        if(l.startswith("###")):
            if(not(propertyName=="")):
                objectProperties[propertyName]=ngsiObjectProperty
                ngsiObjectProperty={}
            propertyName = l[5:len(l)-1]
            ngsiObjectProperty[RDF_TYPE]=OWL_OBJECT_PROPERTY   
        if(l.find(RDFS_SUBPROPERTY_OF)>-1):
            elements = l.split()
            ngsild_type = uri+elements[1][1:]         
            ngsiObjectProperty[NGSILD_TYPE]=ngsild_type
        if(l.find(RDFS_DOMAIN)>-1):
            elements = l.split()
            if(elements[1]=='['):
                readDomain = True
            else:
                ngsiObjectProperty[RDFS_DOMAIN]=[elements[1]]
        if(readDomain and (l.find(OWL_UNION_OF)>-1)):
            elements = l.split()
            domains = []
            prefix=elements[2][0:elements[2].find(':')+1]
            uri = prefixes[prefix]
            uri = uri[1:len(uri)-1]
            fullname = uri+removeprefix(elements[2],prefix)
            domains.append(fullname)
            readUnionOf = True
        elif(readDomain and readUnionOf):
            if(l.find(")")>-1):
                readUnionOf = False
                readDomain = False
                ngsiObjectProperty[RDFS_DOMAIN]=domains
            else:
                elements = l.split()
                prefix=elements[0][0:elements[0].find(':')+1]
                uri = prefixes[prefix]
                uri = uri[1:len(uri)-1]
                fullname = uri+removeprefix(elements[0],prefix)
                domains.append(fullname)                
        if(l.find(RDFS_RANGE)>-1):
            elements = l.split()
            if(elements[1]=='['):
                readRange = True
            else:
                ngsiObjectProperty[RDFS_RANGE]=[elements[1]]
        if(readRange and (l.find(OWL_UNION_OF)>-1)):
            elements = l.split()
            ranges = []
            prefix=elements[2][0:elements[2].find(':')+1]
            uri = prefixes[prefix]
            uri = uri[1:len(uri)-1]
            fullname = uri+removeprefix(elements[2],prefix)
            ranges.append(fullname)
            readUnionOf = True
        elif(readRange and readUnionOf):
            if(l.find(")")>-1):
                readUnionOf = False
                readRange = False
                ngsiObjectProperty[RDFS_RANGE]=ranges
            else:
                elements = l.split()
                prefix=elements[0][0:elements[0].find(':')+1]
                uri = prefixes[prefix]
                uri = uri[1:len(uri)-1]
                fullname = uri+removeprefix(elements[0],prefix)
                ranges.append(fullname)                    
    if(not(propertyName=="")):
        objectProperties[propertyName]=ngsiObjectProperty    
    #for propertyName in objectProperties:
    #    print(propertyName+str(objectProperties[propertyName]))

def parseNGSILDDataProperties():
    global ngsildDatatypeProperties
    global datatypeProperties
    
    ngsiDataProperty = {}
    propertyName = ""
    subelements=[]
    for l in ngsildDatatypeProperties:
        if(l.startswith("###")):
            if(not(propertyName=="")):
                datatypeProperties[propertyName]=ngsiDataProperty
                ngsiDataProperty={}
            propertyName = l[5:len(l)-1]
            ngsiDataProperty[RDF_TYPE]=OWL_DATATYPE_PROPERTY   
        if(l.find(RDFS_SUBPROPERTY_OF)>-1):
            elements = l.split()
            ngsiDataProperty[RDFS_SUBPROPERTY_OF]=elements[1]
        if(l.find(RDFS_DOMAIN)>-1):
            elements = l.split()
            ngsiDataProperty[RDFS_DOMAIN]=[elements[1]]
        if(l.find(RDFS_RANGE)>-1):
            elements = l.split()
            ngsiDataProperty[RDFS_RANGE]=[elements[1]]
    if(not(propertyName=="")):
        datatypeProperties[propertyName]=ngsiDataProperty            
            
    #for propertyName in datatypeProperties:
    #    print(propertyName+str(datatypeProperties[propertyName]))
        

def readNGSILDBaseOntology():
    global prefixesRead
    global ngsildClasses
    global ngsildObjectProperties
    global ngsildDatatypeProperties
    prefixesRead = []
    ngsildClasses = []
    ngsildObjectProperties = []
    ngsildDatatypeProperties = []
    with open('NGSI-LD-Base-Ontology-v1.4.ttl') as f:
        state = "prefix"
        lines = f.readlines()
        for l in lines:
            if l.startswith('#'):
                if "Object Properties" in l:
                    state = "object"
                    continue
                elif "Data" in l:
                    state = "data"
                    continue
                elif "Class" in l:
                    state = "class"
                    continue
                elif l.startswith('############'):
                    continue
            if(state=="prefix"):
                if 'http' in l:
                    prefixesRead.append(l)
            elif(state=="object"):
                ngsildObjectProperties.append(l)
            elif(state=="data"):
                ngsildDatatypeProperties.append(l)
            elif(state=="class"):
                ngsildClasses.append(l)

readNGSILDBaseOntology()
for l in prefixesRead:
    print(l, end='')
parsePrefixes()
for l in inversePrefixes:
    print(l, inversePrefixes[l])

print("")
parseNGSILDClasses()
parseNGSILDObjectProperties()
parseNGSILDDataProperties()


@prefix : <https://uri.etsi.org/ngsi-ld/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix geojson: <https://purl.org/geojson/vocab#> .
@base <https://uri.etsi.org/ngsi-ld/> .
<https://uri.etsi.org/ngsi-ld/> rdf:type owl:Ontology .
https://uri.etsi.org/ngsi-ld/ :
http://www.w3.org/2002/07/owl# owl:
http://www.w3.org/1999/02/22-rdf-syntax-ns# rdf:
http://www.w3.org/XML/1998/namespace xml:
http://www.w3.org/2001/XMLSchema# xsd:
http://www.w3.org/2000/01/rdf-schema# rdfs:
https://purl.org/geojson/vocab# geojson:
https://smartdatamodels.org/ smart:
https://uri.fiware.org/ns/data-models# fiware:
https://smart-data-models.github.io/data-models/terms.jsonld#/definitions/ terms:
https://smart-data-models.github.io/data-models/common-schema.json#/definitions/ definiti

### List of Smart Data Models
* Read file with list of Smart Data Models (created in August 2021) and create URLs
* TODO: find a way to create list directly from github

In [11]:
def readSmartDataModelURLs():
    smartDataModelURLs = []
#    with open('SmartDataModelsList-extract.txt') as f:
    with open('SmartDataModelsList.txt') as f:
#    with open('SmartDataModelsList-WithoutEnergyCIM.txt') as f:
#    with open('SmartDataModelsList-EnergyCIM.txt') as f:
        lines = f.readlines()
        for l in lines:
            pair = l.split()
            url = "https://raw.githubusercontent.com/smart-data-models/" + pair[1] + "/master/" + pair[0] + "/schema.json"
            smartDataModelURLs.append(url)
    return smartDataModelURLs

urls = readSmartDataModelURLs()
for url in urls:
    print(url)

https://raw.githubusercontent.com/smart-data-models/dataModel.Aeronautics/master/AircraftModel/schema.json
https://raw.githubusercontent.com/smart-data-models/dataModel.Aeronautics/master/Aircraft/schema.json
https://raw.githubusercontent.com/smart-data-models/dataModel.Aeronautics/master/Airline/schema.json
https://raw.githubusercontent.com/smart-data-models/dataModel.Aeronautics/master/Airport/schema.json
https://raw.githubusercontent.com/smart-data-models/dataModel.Aeronautics/master/FlightNotification/schema.json
https://raw.githubusercontent.com/smart-data-models/dataModel.Aeronautics/master/Flight/schema.json
https://raw.githubusercontent.com/smart-data-models/dataModel.Agrifood/master/AgriApp/schema.json
https://raw.githubusercontent.com/smart-data-models/dataModel.Agrifood/master/AgriCrop/schema.json
https://raw.githubusercontent.com/smart-data-models/dataModel.Agrifood/master/AgriFarm/schema.json
https://raw.githubusercontent.com/smart-data-models/dataModel.Agrifood/master/Agr

# Read Smart Data Models
### Functionality for reading Smart Data Model JSON schema properties
* read properties from Smart Data Model JSON schema

In [12]:
def getProperties(pathlist, element):
    if not pathlist:
        return element
    else:
        pathelement = pathlist.pop(0)
        #print("Path element: " + pathelement)
        #print("Path list: " + str(pathlist))
        if(pathelement in element):
            return getProperties(pathlist, element[pathelement])
        else:
            return {}

In [13]:
def createComplexType(className, propertyValues):
    global classes
    global datatypeProperties

    classElements =  {}  
    classElements[RDF_TYPE] = OWL_CLASS
    classElements[RDFS_SUBCLASS_OF] = NGSILD_VALUE
    if DESCRIPTION in propertyValues:
        classElements[DESCRIPTION] = propertyValues[DESCRIPTION].replace("Property.", "Value.", 1)
    complexTypeProperties = ""
    if PROPERTIES in propertyValues:
        complexTypeProperties = propertyValues[PROPERTIES]
    elif VALUES in propertyValues:
        complexTypeProperties = propertyValues[VALUES]
    if complexTypeProperties != "":
        local_properties = []
        local_properties.append(complexTypeProperties)
        readProperties(className, local_properties, False)
        classes[className] = classElements
 

In [14]:
def readProperties(entityType, propertyList, isNGSILD_Attribute):
    global objectProperties
    global datatypeProperties
    global classes
    for localproperties in propertyList:   
        propertyNames = localproperties.keys()
        for key in propertyNames:
            #print("Key: " + key)
            propertyValues = localproperties[key]
            if key != 'type':   
                # unless the property key is 'type' get full name of property
                
                ##### expand attribute name to full URI
                if(key in context):
                    value = context[key]
                    if isinstance(value, dict):
                        value=value['@id']                        
                        #print('Relationship: ' + value + ' required: ' + str((key in propertiesRequired)))
                else:
                    value = defaultURL + key
                
                if(value in URI_TRANSLATION):
                     value = URI_TRANSLATION[value]
                
                ###### check whether property has already been defined as Object Property or Datatype Property
                # if property is already contained in objectProperties, read it from there
                inDatatypeProperties = False
                inObjectProperties = False
                
                if value in objectProperties:
                    propertyElements = objectProperties[value]
                    inObjectProperties = True
                # if property is already contained in datatypeProperties, read it from there
                elif value in datatypeProperties:
                    propertyElements = datatypeProperties[value]
                    inDatatypeProperties = True
                else:
                    propertyElements =  {}  
                
                entityRange = None
                ####### check if it is an NGSI-LD relationship
                if key.startswith('ref') and not key.startswith('reference'):
                    # TODO check if type already exists and is set correctly
                    if isNGSILD_Attribute:
                        propertyElements[NGSILD_TYPE]=NGSILD_RELATIONSHIP
                    propertyElements[OWL_PROPERTY] = OWL_OBJECT_PROPERTY
                    entityRange = usePrefix(ENTITY_URI)
                    propertyElements[RDFS_RANGE] = [entityRange]
                    
                ###### it has to be an NGSI-LD property, if its value is simple, it is an OWL datatype property
                ###### otherwise it is an OWL object property
                else:
                    if isNGSILD_Attribute:
                        propertyElements[NGSILD_TYPE]=NGSILD_PROPERTY
                    ## TODO check for existance of 'type'
                    if TYPE in propertyValues:
                        propertyType = propertyValues[TYPE]
                        if propertyType in JSON_SIMPLE_TYPES:
                            propertyElements[OWL_PROPERTY] = OWL_DATATYPE_PROPERTY                                    
                            if propertyType in JSON_SIMPLE_TYPES:
                                propertyElements[RDFS_RANGE] = [TYPE_TRANSLATION[propertyType]]        
                                if FORMAT in propertyValues:
                                    format = propertyValues[FORMAT]
                                    if format == DATE_TIME:
                                        propertyElements[RDFS_RANGE] = [XSD_DATETIME]
                                if (propertyType == ARRAY) or (propertyType == STRING):
                                    # print("Key: " + key + " Property type: " + propertyType)
                                    if ITEMS in propertyValues:
                                        # print("Key: " + key)
                                        # print("  Type: " + propertyType)
                                        items = propertyValues[ITEMS]
                                        if isinstance(items, list):
                                            items = items[0]
                                        if DOLLAR_REF in items:
                                            ref_uri = items[DOLLAR_REF]
                                            # print("-- Reference URI: " + ref_uri)
                                            if ref_uri == ENTITY_IDENTIFIER_URI:
                                                if isNGSILD_Attribute:
                                                    propertyElements[NGSILD_TYPE] = NGSILD_RELATIONSHIP
                                                propertyElements[OWL_PROPERTY] = OWL_OBJECT_PROPERTY
                                                entityRange = usePrefix(ENTITY_URI)
                                                propertyElements[RDFS_RANGE] = [entityRange]
                                            else:
                                                propertyElements[ITEM_TYPE] = usePrefix(ref_uri)
                                            # print( "  Item type: " + propertyElements[ITEM_TYPE])
                                        else:
                                            if TYPE in items:
                                                if items[TYPE] in TYPE_TRANSLATION:
                                                    propertyElements[ITEM_TYPE] = TYPE_TRANSLATION[items[TYPE]]
                                                    # print( "  Item type: " + propertyElements[ITEM_TYPE])
                                                    if FORMAT in items:
                                                        item_format = TYPE_TRANSLATION[items[TYPE]]
                                                        if item_format == DATE_TIME:
                                                            propertyElements[ITEM_TYPE] = XSD_DATETIME
                                                            # print( "  REPLACED: Item type: " + propertyElements[ITEM_TYPE])
                                                        else:
                                                            propertyElements[ITEM_FORMAT] = item_format
                                                        # print( "  Item format: " + propertyElements[ITEM_FORMAT])
                                                    if ENUM in items:
                                                        propertyElements[ITEM_ENUM] = items[ENUM]
                                                        # print( "  Item enum: " + str(propertyElements[ITEM_ENUM]))
                                                elif (items[TYPE]==OBJECT):
                                                    if isNGSILD_Attribute:
                                                        propertyElements[NGSILD_TYPE] = NGSILD_PROPERTY
                                                    propertyElements[OWL_PROPERTY] = OWL_OBJECT_PROPERTY
                                                    propertyKey = defaultURL + key + TYPE_POSTFIX
                                                    if (not (propertyKey in classes)): # for now take first definition only
                                                        if VALUES in items:
                                                            #print("Type " + propertyKey + " has to be created")
                                                            createComplexType(propertyKey, items)
                                                    propertyElements[RDFS_RANGE] = [propertyKey]
                                                else:    
                                                    print("Key: " + key + "  Unknown item type: " + items[TYPE])
                                            elif ANY_OF in items:
                                                #print("ANY OF DETECTED: " +  key)
                                                anyof = str(items[ANY_OF])
                                                if anyof.find(ID_FORMAT) > 0:
                                                    #print("--ID  FORMAT--")
                                                    if (DESCRIPTION in propertyValues):
                                                        anyof_description = propertyValues[DESCRIPTION]
                                                        if anyof_description.find(RELATIONSHIP) > -1:
                                                            #print("--RELATIONSHIP--")
                                                            if isNGSILD_Attribute:
                                                                propertyElements[NGSILD_TYPE] = NGSILD_RELATIONSHIP
                                                                propertyElements[OWL_PROPERTY] = OWL_OBJECT_PROPERTY
                                                                entityRange = usePrefix(ENTITY_URI)
                                                                propertyElements[RDFS_RANGE] = [entityRange]
                                                            else:
                                                                print("--URI-->DATATYPE_PROPERTY--")
                                                                propertyElements[OWL_PROPERTY] = OWL_DATATYPE_PROPERTY
                                                                propertyElements[RDFS_RANGE] = [XSD_ANYURI]
                        else:
                            propertyElements[OWL_PROPERTY] = OWL_OBJECT_PROPERTY
                            # add range - check if there can be multiple ranges ...                        
                            if propertyType == OBJECT:
                                propertyKey = defaultURL + key + TYPE_POSTFIX
                                if (not (propertyKey in classes)): # for now take first definition only
                                    if PROPERTIES in propertyValues:
                                        # print("Type " + propertyKey + " has to be created")
                                        createComplexType(propertyKey, propertyValues)
                                    else:
                                        model = propertyValues[DESCRIPTION]
                                        element =  model.find("Model:'")
                                        if element > 0:
                                            model = model[element + 7:]
                                            element = model.find("'")
                                            if element > 0:
                                                propertyKey = model[:element]
                                propertyElements[RDFS_RANGE] = [propertyKey]
                            else: 
                                print("################## What happens here?")
                                print(" Unexpected Property Type:" + str(propertyType) + " instead of Object")
                    elif (key == ID):
                        continue
                    elif (key == LOCATION):
                        continue
                    elif (key == SPEED):
                            if isNGSILD_Attribute:
                                propertyElements[NGSILD_TYPE] = NGSILD_PROPERTY
                            propertyElements[OWL_PROPERTY] = OWL_DATATYPE_PROPERTY
                            propertyElements[RDFS_RANGE] = [TYPE_TRANSLATION[NUMBER]]
                    elif (key == HEADING):
                            if isNGSILD_Attribute:
                                propertyElements[NGSILD_TYPE] = NGSILD_PROPERTY
                            propertyElements[OWL_PROPERTY] = OWL_DATATYPE_PROPERTY
                            propertyElements[RDFS_RANGE] = [TYPE_TRANSLATION[NUMBER]]                        
                    elif (key == SEE_ALSO):
                        if value in datatypeProperties:
                            continue
                        else:
                            propertyElements[OWL_PROPERTY] = OWL_DATATYPE_PROPERTY
                            propertyElements[RDFS_RANGE] = [XSD_ANYURI, RDF_LIST]
                            propertyElements[ITEM_TYPE] = XSD_ANYURI
                    elif (ONE_OF in propertyValues):
                        oneof = str(propertyValues[ONE_OF])
                        if oneof.find(ENTITY_IDENTIFIER_URI) > 0:
                            if isNGSILD_Attribute:
                                propertyElements[NGSILD_TYPE] = NGSILD_RELATIONSHIP
                            propertyElements[OWL_PROPERTY] = OWL_OBJECT_PROPERTY
                            entityRange = usePrefix(ENTITY_URI)
                            propertyElements[RDFS_RANGE] = [entityRange]
                        else:
                            continue
                    elif (ANY_OF in propertyValues):
                        #print("ANY OF DETECTED: " +  key)
                        anyof = str(propertyValues[ANY_OF])
                        if anyof.find(ID_FORMAT) > 0:
                            #print("--ID  FORMAT--")
                            if (DESCRIPTION in propertyValues):
                                anyof_description = propertyValues[DESCRIPTION]
                                if anyof_description.find(RELATIONSHIP) > -1:
                                    #print("--RELATIONSHIP--")
                                    if isNGSILD_Attribute:
                                        propertyElements[NGSILD_TYPE] = NGSILD_RELATIONSHIP
                                    propertyElements[OWL_PROPERTY] = OWL_OBJECT_PROPERTY
                                    entityRange = usePrefix(ENTITY_URI)
                                    propertyElements[RDFS_RANGE] = [entityRange]
                                else:
                                    print("--URI-->DATATYPE_PROPERTY--")
                                    propertyElements[OWL_PROPERTY] = OWL_DATATYPE_PROPERTY
                                    propertyElements[RDFS_RANGE] = [XSD_ANYURI]
                        else:
                            print("################## What happens here?")
                            for element in propertyValues:
                                print(key+ ": " +str(element))
                            continue
                    elif (DOLLAR_REF in propertyValues):
                        reference = propertyValues[DOLLAR_REF]
                        if not(reference in definitions) and (reference != ENTITY_IDENTIFIER_URI) and (reference != LOCATION_URI):
                            print("### Warning: definition for " + key + " not included! " + reference)
                        if reference == ENTITY_IDENTIFIER_URI:
                            propertyElements[OWL_PROPERTY] = OWL_OBJECT_PROPERTY
                            propertyElements[RDFS_RANGE] = [usePrefix(ENTITY_URI)]
                        else:
                            propertyElements[OWL_PROPERTY] = OWL_DATATYPE_PROPERTY 
                            propertyElements[RDFS_RANGE] = [usePrefix(reference)]
                    else:
                        print("################## What happens here?")
                        for element in propertyValues:
                            print(key+ ": " +str(element))
                            
                        continue
            
                #### adding domains
                domain = []
                if RDFS_DOMAIN in propertyElements:
                    domain = propertyElements[RDFS_DOMAIN].copy()
                if not (entityType in domain):
                    domain.append(entityType)
                propertyElements[RDFS_DOMAIN] = domain

                #### adding description
                if DESCRIPTION in propertyValues:
                    propertyElements[RDFS_COMMENT] = propertyValues[DESCRIPTION]
                
                ##### add propertyElements to objectProperties or datatypeProperties
                if propertyElements[OWL_PROPERTY] == OWL_DATATYPE_PROPERTY:
                    datatypeProperties[value] = propertyElements
                    if inObjectProperties:
                        print("WARINING: " + entityType + " - " + value + " also contained in object properties!!!")                            
                    #if value == "https://smart-data-models.github.io/data-models/terms.jsonld#/definitions/hasAgriSoil":
                    #    print("WARINING: " + entityType + " - " + value)
                else:
                    objectProperties[value] = propertyElements    
                    if inDatatypeProperties:
                        print("WARINING: " + entityType + " - " + value + " also contained in datatype properties!!!")
                        entityRange = usePrefix(ENTITY_URI)
                        if RDFS_RANGE in propertyElements:
                            if propertyElements[RDFS_RANGE] == [entityRange]:
                                datatypeProperties.pop(value, None)
                                print("   Removed element with range :Entity from datatype properties.")
                    #if value == "https://smart-data-models.github.io/data-models/terms.jsonld#/definitions/hasAgriSoil":
                    #    print("WARINING: " + entityType + " - " + value)


* read definitions from Smart Data Model JSON schema

In [15]:
def readDefinition(jsonSchema):
    owlRepresentation = {}
    for key in jsonSchema:
        if key == TYPE:
            #owlRepresentation[RDF_TYPE] = RDFS_DATATYPE
            owlRepresentation[OWL_ON_DATATYPE] = TYPE_TRANSLATION[jsonSchema[TYPE]]
        elif key == FORMAT:
            if jsonSchema[FORMAT] == URI:
                owlRepresentation[OWL_ON_DATATYPE] = TYPE_TRANSLATION[URI]
            elif jsonSchema[FORMAT] == DATE_TIME:
                owlRepresentation[OWL_ON_DATATYPE] = TYPE_TRANSLATION[DATE_TIME]
            else:
                description = jsonSchema[FORMAT]
                if RDFS_COMMENT in owlRepresentation:
                    description = description + ". " + owlRepresentation[RDFS_COMMENT]
                owlRepresentation[RDFS_COMMENT] = description
        elif key == ITEMS:
            ### TODO: fix
            continue
        elif key == ENUM:
            ### TODO: fix
            continue            
        else:
            value = jsonSchema[key]
            if TYPE_TRANSLATION[key] == RDFS_COMMENT:
                if RDFS_COMMENT in owlRepresentation:
                    value = owlRepresentation[RDFS_COMMENT] + ". " + value
            owlRepresentation[TYPE_TRANSLATION[key]] = value
    return owlRepresentation

def readDefinitions(definitionList):
    global definitions
    definitionNames = definitionList.keys();
    for name in definitionNames:
        if name == ENTITY_IDENTIFIER_TYPE:
            continue
        anyOf = False
        #print("Definition: " + name)
        localDefinition = definitionList[name]
        if TYPE in localDefinition:
            if localDefinition[TYPE] == OBJECT:
                continue
        else:
            if not(ANY_OF in localDefinition):
                continue
            else:
                anyOf = True
            
        ##### expand attribute name to full URI
        if(name in context):
            uri = context[name]
            if isinstance(uri, dict):
                value=value['@id'] 
        else:
            uri = defaultURL + name
        
        if(uri in URI_TRANSLATION):
            uri = URI_TRANSLATION[uri]
      
        if anyOf:
            #print("Definition -anyOf : " + uri)   
            elements = []
            for element in localDefinition[ANY_OF]:
                elements.append(readDefinition(element))
            definitions[uri] = elements
        else:
            #print("Definition: " + uri)    
            definitions[uri] = readDefinition(localDefinition)
        
        

## Read Smart Data Model

In [16]:
def readSmartDataModel(data):
    global includes
    global classes
    global properties
    local_properties = []
    if not (ALL_OF in data):
        if DOLLAR_ID in data:
            id = data[DOLLAR_ID]
        elif TITLE in data:
            id = data[TITLE]
        elif DESCRIPTION in data:
            id = data[DESCRIPTION]
        else:
            id = "UNKNOWN"
        print("Error in " + id + " - " + ALL_OF + " not present.")
        return
    allOf = data[ALL_OF]
    for element in allOf:
        for item in element:
            if(item==DOLLAR_REF):
                reference = element[item]
                #print("Reference: " + reference)
                if reference.find("#") > -1:
                    referenceparts = reference.split('#')
                    url = referenceparts[0]
                    path = referenceparts[1]
                else:
                    url = reference
                    path = ""
                #print('URL: ' + url)
                #print('Path: ' + path)
                if(not(url in includes)):
                    external=readFromURL(url)
                    includes[url] = external
                    # TODO read definitions and create OWL representations of datatypes
                    #print("External URL " + url + " loaded!")
                    # parse definitions
                    includedDefinitions = includes[url][DEFINITIONS]
                    readDefinitions(includedDefinitions)
                if path != "":
                    pathelements = path.split('/')
                    del pathelements[0]
                    propertiesElement = getProperties(pathelements, includes[url])
                    if PROPERTIES in propertiesElement:
                        propertiesObject = propertiesElement[PROPERTIES]
                        local_properties.append(propertiesObject)
            elif(item==PROPERTIES):
                local_properties.append(element[item])
                propertyValues = element[item]
                #print(element[item])
            else:
                print('Unknown element: ' + item)
        
    # TODO put in classes
    if not (TYPE in propertyValues):
        print("ERROR: undefined type, classname cannot be extracted")
        return
    propertyType = propertyValues[TYPE]
    if ENUM in propertyType:
        shortClassName = propertyType[ENUM][0]
    elif VALUE in propertyType:
        shortClassName = propertyType[VALUE]
    else:
        print("ERROR: undefined type, classname cannot be extracted")
        return
    if shortClassName in context:
        className = context[shortClassName]
    else:
        className = defaultURL + shortClassName
    owlclass = {}
    owlclass[RDF_TYPE] = OWL_CLASS
    owlclass[RDFS_SUBCLASS_OF] = NGSILD_ENTITY
    if REQUIRED in data:
        restrictions = data[REQUIRED]
        owl_restrictions = []
        for restriction in restrictions:
            if(restriction in context):
                owl_restriction = context[restriction]
                if isinstance(owl_restriction, dict):
                    owl_restriction=owl_restriction['@id']                        
                    #print('Relationship: ' + value + ' required: ' + str((key in propertiesRequired)))
            else:
                owl_restriction = defaultURL + restriction
            if not(owl_restriction == AT_ID or owl_restriction == AT_TYPE):
                owl_restrictions.append(owl_restriction)
        if len(owl_restrictions)>0:
            owlclass[OWL_RESTRICTION] = owl_restrictions
        #print("OWL Restrictions: " + str(owl_restrictions))
    classes[className] = owlclass
    readProperties(className, local_properties, True)
    properties.append(local_properties)


# Print Ontology
* function for printing defintions
* function for printing object properties
* function for prinding datatype properties
* helper function for extracting name from URL
* helper function for using prefix
* function for printing ontology

In [17]:
def printRestrictions(value, prefixLength):
    for element in value:
        if((element in RESTRICTIONS)):
            if(element==XSD_PATTERN):
                print(" " * prefixLength + "[ " + element + " \"" + str(value[element])+"\" ]")
            else:
                print(" " * prefixLength + "[ " + element + " " + str(value[element])+" ]")
        

def printDefinitionDetails(value, prefixLength, noFullstop=None):
    restriction = False
    numberOfElements = len(value.keys())
    i=0
    if(noFullstop!=None):
        print(" " * prefixLength + " rdf:type rdfs:Datatype ;")
    for element in value:
        i=i+1
        
        if((element in RESTRICTIONS)):
            if (not(restriction)):
                restriction = True
                print(" " * prefixLength + " " + OWL_WITH_RESTRICTIONS + " ( ")
                printRestrictions(value, prefixLength+len(OWL_WITH_RESTRICTIONS) + 5 )
                print(" " * (prefixLength+len(OWL_WITH_RESTRICTIONS)) + "  )", end='')
                if(i < numberOfElements):
                    print(" ;")
                elif(noFullstop!=None):
                     print(" ")
                else:
                    print(" .\n")
        else: 
            if(element==RDFS_COMMENT):
                print(" " * prefixLength + " " + element + " \"" + str(value[element])+"\"", end='')
            else:
                print(" " * prefixLength + " " + element + " " + str(value[element]), end='')
            if(i < numberOfElements):
                print(" ;")
            elif(noFullstop!=None):
                print(" ")
            else:
                print(" .\n")
    
    
def printDefinitions():
    global definitions
    print('#################################################################')
    print('#    Datatypes')
    print('#################################################################\n')
    definitionNames = definitions.keys();
    for key in definitionNames:
        value = definitions[key]
        print('### ' + key)
        shortName = usePrefix(key)
        print( shortName + " rdf:type rdfs:Datatype ;")
        prefixLength = len(shortName)
        if type(value) is list:
            print( " " * prefixLength + " owl:DataUnionOf ( ")
            for listElement in value:
                print( " " * (prefixLength + 18) + " [")
                printDefinitionDetails(listElement, prefixLength+22, "yes")
                print( " " * (prefixLength + 18) + " ]")
            print( " " * prefixLength + " ) .\n")
        else:
            
            printDefinitionDetails(value, prefixLength)
            
        

In [18]:
usePrefix("https://smart-data-models.github.io/data-models/terms.json#/definitions/userAlias")

'https://smart-data-models.github.io/data-models/terms.json#/definitions/userAlias'

In [19]:
def printClasses():
    global classes
    print('#################################################################')
    print('#    Classes')
    print('#################################################################\n')
    for className in classes:
        print("### " + className)
        shortName = usePrefix(className)
        owlClass = classes[className]
        print(shortName + " " + RDF_TYPE + " " + owlClass[RDF_TYPE], end="")
        if(RDFS_SUBCLASS_OF in owlClass):
            print(" ;")
            print(" " * (len(shortName)+1) + RDFS_SUBCLASS_OF + " " + usePrefix(owlClass[RDFS_SUBCLASS_OF]), end="")
        if(OWL_RESTRICTION in owlClass):
            restrictions = owlClass[OWL_RESTRICTION]
            for restriction in restrictions:
                print(" ;")
                print(" " * (len(shortName)+1) + RDFS_SUBCLASS_OF + " [ " + RDF_TYPE + " " + OWL_RESTRICTION + " ;")
                print(" " * (len(shortName) + len (RDFS_SUBCLASS_OF) + 4) + OWL_ON_PROPERTY + " " + usePrefix(restriction) + " ;")
                print(" " * (len(shortName) + len (RDFS_SUBCLASS_OF) + 4) +  OWL_MIN_CARDINALITY_Q + " " + MIN_CARDINALITY_1)
                print(" " * (len(shortName) + len (RDFS_SUBCLASS_OF) + 1) + " ]" , end="")            
        if(OWL_DISJOINT_WITH in owlClass):
            print(" ;")
            owlDisjoint = owlClass[OWL_DISJOINT_WITH]
            if(type(owlDisjoint) == str):
                print(" " * (len(shortName)+1) + OWL_DISJOINT_WITH + " " + owlDisjoint, end="")
            else:
                print(" " * (len(shortName)+1) + OWL_DISJOINT_WITH + " " + owlDisjoint[0], end="")
                if(len(owlDisjoint)>1):
                    for i in range(1,len(owlDisjoint)):
                        print(" ,")
                        print(" " * (len(shortName) + len(OWL_DISJOINT_WITH) + 2) + owlDisjoint[i], end="")
        if(DESCRIPTION in owlClass):
            print(" ;")
            description = owlClass[DESCRIPTION]
            print(" " * (len(shortName)+1) + RDFS_COMMENT + " \"" + description + "\"", end="")
        print(" .")
        print("\n")


In [20]:
printClasses()

#################################################################
#    Classes
#################################################################

### https://uri.etsi.org/ngsi-ld/Entity
:Entity rdf:type owl:Class ;
        owl:disjointWith :Property ,
                         :Relationship ,
                         :Value .


### https://uri.etsi.org/ngsi-ld/Geometry
:Geometry rdf:type owl:Class ;
          rdfs:subClassOf :Value .


### https://uri.etsi.org/ngsi-ld/LanguageMap
:LanguageMap rdf:type owl:Class ;
             rdfs:subClassOf :Value .


### https://uri.etsi.org/ngsi-ld/LineString
:LineString rdf:type owl:Class ;
            rdfs:subClassOf :Geometry .


### https://uri.etsi.org/ngsi-ld/Point
:Point rdf:type owl:Class ;
       rdfs:subClassOf :Geometry .


### https://uri.etsi.org/ngsi-ld/MultiPoint
:MultiPoint rdf:type owl:Class ;
            rdfs:subClassOf :Geometry .


### https://uri.etsi.org/ngsi-ld/Polygon
:Polygon rdf:type owl:Class ;
         rdfs:subClassOf :Geo

In [21]:
def printObjectProperties():
    global objectProperties
    print('#################################################################')
    print('#    Object Properties')
    print('#################################################################\n')
    
    propertyNames = objectProperties.keys();
    for key in propertyNames:    
        print('###  ' + key)
        shortName = usePrefix(key)
        shortNameLength = len(shortName)
        print( shortName + " " + RDF_TYPE  + " " + OWL_OBJECT_PROPERTY, end = "")
        
        if NGSILD_TYPE in objectProperties[key]:
            if objectProperties[key][NGSILD_TYPE] == NGSILD_RELATIONSHIP:
                print(" ;")
                print(" " * shortNameLength + " " + RDFS_SUBPROPERTY_OF + " "+  usePrefix(NGSILD_RELATIONSHIP), end ="")
            elif objectProperties[key][NGSILD_TYPE] == NGSILD_GEOPROPERTY:
                print(" ;")
                print(" " * shortNameLength + " " + RDFS_SUBPROPERTY_OF + " "+  usePrefix(NGSILD_GEOPROPERTY), end = "")
            elif objectProperties[key][NGSILD_TYPE] == NGSILD_PROPERTY:
                print(" ;")
                print(" " * shortNameLength + " " + RDFS_SUBPROPERTY_OF + " "+  usePrefix(NGSILD_PROPERTY), end="")
            elif objectProperties[key][NGSILD_TYPE] == NGSILD_LANGUAGE_PROPERTY:
                print(" ;")
                print(" " * shortNameLength + " " + RDFS_SUBPROPERTY_OF + " "+  usePrefix(NGSILD_LANGUAGE_PROPERTY), end = "")
            else: 
                print(" ;")
                print("# **** UNKNOWN NGSI-LD ATTRIBUTE TYPE **** ")
              
        domain = []
        if RDFS_DOMAIN in objectProperties[key]:
            domain = objectProperties[key][RDFS_DOMAIN]

        if type(domain)==list:
            if len(domain) > 1:
                print(" ;")
                print(" " * (shortNameLength + 1) + RDFS_DOMAIN + " [ " + RDF_TYPE + " " + OWL_CLASS + " ;") 
                print(" " * (shortNameLength + len(RDFS_DOMAIN) + 4) +  OWL_UNION_OF +  " ( " + usePrefix(domain[0]))
                for i in range(len(domain)-1):
                    print(" " * (shortNameLength + len(RDFS_DOMAIN) + len(OWL_UNION_OF) + 7) + usePrefix(domain[i+1]))
                print(" " * (shortNameLength + len(RDFS_DOMAIN) + len(OWL_UNION_OF) + 5) + ")")
                print(" " * (shortNameLength  + len(RDFS_DOMAIN) +2) + "]", end="")
            elif len(domain)==1:
                print(" ;")
                print(" " * shortNameLength + " " + RDFS_DOMAIN + " " + usePrefix(domain[0]), end ="") 

        rdfs_range = []
        if RDFS_RANGE in objectProperties[key]:
            rdfs_range = objectProperties[key][RDFS_RANGE]

        if type(rdfs_range)==list:
            if len(rdfs_range) > 1:
                print(" ;")
                print(" " * (shortNameLength + 1) + RDFS_RANGE + " [ " + RDF_TYPE + " " + OWL_CLASS + " ;") 
                print(" " * (shortNameLength + len(RDFS_RANGE) + 4) +  OWL_UNION_OF +  " ( " + usePrefix(rdfs_range[0]))
                for i in range(len(rdfs_range)-1):
                    print(" " * (shortNameLength + len(RDFS_RANGE) + len(OWL_UNION_OF) + 7) + usePrefix(rdfs_range[i+1]))
                print(" " * (shortNameLength + len(RDFS_RANGE) + len(OWL_UNION_OF) + 5) + ")")
                print(" " * (shortNameLength  + len(RDFS_RANGE) +2) + "]", end="")
            elif len(rdfs_range)==1:
                print(" ;")
                print(" " * shortNameLength + " " + RDFS_RANGE + " " + usePrefix(rdfs_range[0]), end ="") 

        if RDFS_COMMENT in objectProperties[key]:
            print(" ;")
            print(" " * shortNameLength + " rdfs:comment \"" + objectProperties[key][RDFS_COMMENT] + "\"", end="") 
        print(" .\n")
               
def printDatatypeProperties():
    global datatypeProperties
    print('#################################################################')
    print('#    Data Properties')
    print('#################################################################\n')
    
    propertyNames = datatypeProperties.keys();
    for key in propertyNames:
        print('###  ' + key)
        shortName = usePrefix(key)
        shortNameLength = len(shortName)
        print( shortName + " " + RDF_TYPE  + " " + OWL_DATATYPE_PROPERTY, end = "")
        
        if NGSILD_TYPE in datatypeProperties[key]:
            if datatypeProperties[key][NGSILD_TYPE] == NGSILD_PROPERTY:
                print(" ;")
                print(" " * shortNameLength + " " + RDFS_SUBPROPERTY_OF + " "+  usePrefix(NGSILD_PROPERTY), end="")
            else: 
                print(" ;")
                print("# **** UNKNOWN NGSI-LD PROPERTY TYPE **** ")
          
        domain = []
        if RDFS_DOMAIN in datatypeProperties[key]:
            domain = datatypeProperties[key][RDFS_DOMAIN]

        if type(domain)==list:
            if len(domain) > 1:
                print(" ;")
                print(" " * (shortNameLength + 1) + RDFS_DOMAIN + " [ " + RDF_TYPE + " " + OWL_CLASS + " ;") 
                print(" " * (shortNameLength + len(RDFS_DOMAIN) + 4) +  OWL_UNION_OF +  " ( " + usePrefix(domain[0]))
                for i in range(len(domain)-1):
                    print(" " * (shortNameLength + len(RDFS_DOMAIN) + len(OWL_UNION_OF) + 7) + usePrefix(domain[i+1]))
                print(" " * (shortNameLength + len(RDFS_DOMAIN) + len(OWL_UNION_OF) + 5) + ")")
                print(" " * (shortNameLength  + len(RDFS_DOMAIN) +2) + "]", end="")
            elif len(domain)==1:
                print(" ;")
                print(" " * shortNameLength + " " + RDFS_DOMAIN + " " + usePrefix(domain[0]), end ="") 

        rdfs_range = []
        if RDFS_RANGE in datatypeProperties[key]:
            rdfs_range = datatypeProperties[key][RDFS_RANGE]

        if type(rdfs_range)==list:
            if len(rdfs_range) > 1:
                print(" ;")
                print(" " * (shortNameLength + 1) + RDFS_RANGE + " [ " +  RDF_TYPE + " " + RDFS_DATATYPE + " ;") 
                print(" " * (shortNameLength + len(RDFS_RANGE) + 4) + OWL_UNION_OF +  " ( " + usePrefix(rdfs_range[0]))
                for i in range(len(rdfs_range)-1):
                    print(" " * (shortNameLength + len(RDFS_RANGE) + len(OWL_UNION_OF) + 7) + usePrefix(rdfs_range[i+1]))
                print(" " * (shortNameLength + len(RDFS_RANGE) + len(OWL_UNION_OF) + 5) + " )")
                print(" " * (shortNameLength + len(RDFS_RANGE) + 2) + " ]", end="")
            elif len(rdfs_range)==1:
                if (usePrefix(rdfs_range[0]) == XSD_STRING) and (ITEM_ENUM in datatypeProperties[key]):
                    enum = datatypeProperties[key][ITEM_ENUM]                    
                    print(" ;")
                    print(" " * (shortNameLength + 1) + RDFS_RANGE + " " + XSD_STRING + " ,")
                    print(" " * (shortNameLength + len(RDFS_RANGE) + 2) + "[ " + RDF_TYPE + " " + RDFS_DATATYPE + " ;")
                    print(" " * (shortNameLength + len(RDFS_RANGE) + 4) + OWL_ONE_OF + " [ ", end="")
                    printEnum(enum, shortNameLength + len(RDFS_RANGE) + len (OWL_ONE_OF) + 7)
                    print(" " * (shortNameLength + len(RDFS_RANGE) + len(OWL_ONE_OF) + 4)  + " ]")
                    print(" " * (shortNameLength + len(RDFS_RANGE) + 2) + "] ", end="")
                else:    
                    print(" ;")
                    print(" " * shortNameLength + " " + RDFS_RANGE + " " + usePrefix(rdfs_range[0]), end ="") 

        comments = ""
        if RDFS_COMMENT in datatypeProperties[key]:
            comments = datatypeProperties[key][RDFS_COMMENT]
        
        if ITEM_TYPE in datatypeProperties[key]:
            comments = comments + " -- " + ITEM_TYPE + ":" + datatypeProperties[key][ITEM_TYPE]
            if ITEM_FORMAT in datatypeProperties[key]:
                comments = comments + ", " + ITEM_FORMAT + ":" + datatypeProperties[key][ITEM_FORMAT]
            if ITEM_ENUM in datatypeProperties[key]:
                comments = comments + ", " + ITEM_ENUM + ":" + str(datatypeProperties[key][ITEM_ENUM])
            comments = comments + "."
        
        if comments != "":
            print(" ;")
            print(" " * shortNameLength + " rdfs:comment \"" + comments + "\"", end="") 
        print(" .\n")

def printEnum(enum,spaces):
        first = enum[0]
        if len(enum) > 1:
            rest = enum[1:]
        else:
            rest = ""
        print(RDF_TYPE + " " + RDF_LIST+ " ;")
        print(" " * spaces + RDF_FIRST + " \"" + first + "\"^^" + XSD_STRING + " ;")
        if (rest != ""):
            print(" " * spaces + RDF_REST + " [ ", end = "")
            printEnum(rest,spaces + len(RDF_REST) + 3)
            print(" " * (spaces + len(RDF_REST)) + " ] ")
        else:
            print(" " * spaces + RDF_REST + " " + RDF_NIL)
        


In [22]:
printObjectProperties()

#################################################################
#    Object Properties
#################################################################

###  https://uri.etsi.org/ngsi-ld/GeoProperty
:GeoProperty rdf:type owl:ObjectProperty ;
             rdfs:subPropertyOf :Property ;
             rdfs:range :Geometry .

###  https://uri.etsi.org/ngsi-ld/LanguageProperty
:LanguageProperty rdf:type owl:ObjectProperty ;
                  rdfs:subPropertyOf :Property ;
                  rdfs:range :LanguageMap .

###  https://uri.etsi.org/ngsi-ld/Property
:Property rdf:type owl:ObjectProperty ;
          rdfs:domain [ rdf:type owl:Class ;
                        owl:unionOf ( :Entity
                                      :Property
                                      :Relationship
                                    )
                      ] ;
          rdfs:range [ rdf:type owl:Class ;
                       owl:unionOf ( :Property
                                     :Value
        

In [23]:
printDatatypeProperties()

#################################################################
#    Data Properties
#################################################################

###  https://uri.etsi.org/ngsi-ld/TemporalProperty
:TemporalProperty rdf:type owl:DatatypeProperty .

###  https://uri.etsi.org/ngsi-ld/createdAt
:createdAt rdf:type owl:DatatypeProperty ;
           rdfs:range xsd:dateTime .

###  https://uri.etsi.org/ngsi-ld/hasValue
:hasValue rdf:type owl:DatatypeProperty ;
          rdfs:domain :Property .

###  https://uri.etsi.org/ngsi-ld/hasLanguageMap
:hasLanguageMap rdf:type owl:DatatypeProperty ;
                rdfs:domain :LanguageProperty ;
                rdfs:range :LanguageMap .

###  https://uri.etsi.org/ngsi-ld/modifiedAt
:modifiedAt rdf:type owl:DatatypeProperty ;
            rdfs:range xsd:dateTime .

###  https://uri.etsi.org/ngsi-ld/observedAt
:observedAt rdf:type owl:DatatypeProperty ;
            rdfs:range xsd:dateTime .

###  https://uri.etsi.org/ngsi-ld/unitCode
:unitCode rd

In [24]:
def printPrefixes(): # print prefixes
    global prefixes
    global baseinfo
    for l in prefixes:
        print("@prefix " + l + " " + prefixes[l] + " .")
    for l in baseInfo:
        print(l)

In [25]:
printPrefixes()

@prefix : <https://uri.etsi.org/ngsi-ld/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix geojson: <https://purl.org/geojson/vocab#> .
@prefix smart: <https://smartdatamodels.org/> .
@prefix fiware: <https://uri.fiware.org/ns/data-models#> .
@prefix terms: <https://smart-data-models.github.io/data-models/terms.jsonld#/definitions/> .
@prefix definitions: <https://smart-data-models.github.io/data-models/common-schema.json#/definitions/> .
@prefix common: <https://smart-data-models.github.io/data-models/common-schema.json#/definitions/Location-Commons/properties/> .
@prefix saref: <https://w3id.org/def/saref#> .
@prefix saref4agri: <https://w3id.org/def/saref4agri#> .
@prefix goodrelations: <http://purl.org/goodrelations/v1#> .
@prefix purlterms: <http://purl.

In [26]:
import sys
def printOntology(filename=None):
    f = sys.stdout
    # if the ontology should be written to a file, open file and change stdout to file
    if(filename!=None):
        print("Printing to file: " + filename)
        original_stdout = sys.stdout # Save a reference to the original standard output
        f = open(filename, 'w')
        sys.stdout = f # Change the standard output to the file we created.
        
    printPrefixes()
    printDefinitions()
    printClasses()
    printObjectProperties()
    printDatatypeProperties()
    
    # if the ontology was written to a file, close file and restore stdout
    if(filename!=None):
        f.close
        sys.stdout = original_stdout # Reset the standard output to its original value
        print("Printed to file: " + filename)

In [27]:
printOntology()

@prefix : <https://uri.etsi.org/ngsi-ld/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix geojson: <https://purl.org/geojson/vocab#> .
@prefix smart: <https://smartdatamodels.org/> .
@prefix fiware: <https://uri.fiware.org/ns/data-models#> .
@prefix terms: <https://smart-data-models.github.io/data-models/terms.jsonld#/definitions/> .
@prefix definitions: <https://smart-data-models.github.io/data-models/common-schema.json#/definitions/> .
@prefix common: <https://smart-data-models.github.io/data-models/common-schema.json#/definitions/Location-Commons/properties/> .
@prefix saref: <https://w3id.org/def/saref#> .
@prefix saref4agri: <https://w3id.org/def/saref4agri#> .
@prefix goodrelations: <http://purl.org/goodrelations/v1#> .
@prefix purlterms: <http://purl.

# Main Function
* Decide which smart data models to include
* Read smart data models
* Print ontology with the selected smart data models including the NGSI-LD Base Ontology

In [28]:
includes = {}
definitions = {}
namespaces = {}
reverse_namespeace = {}

classes = {}
properties = []
objectProperties = {}
datatypeProperties = {}
ngsildClasses = []
ngsildObjectProperties = []
ngsildDatatypeProperties = []


defaultURL = "https://uri.etsi.org/ngsi-ld/default-context/"
parsePrefixes()

readNGSILDBaseOntology()
parseNGSILDClasses()
parseNGSILDObjectProperties()
parseNGSILDDataProperties()

context = {}
core_context = {}

core_context_url = 'https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context-v1.8.jsonld'

smart_data_model_context_urls = [ 'https://smart-data-models.github.io/data-models/context.jsonld' ]

## new links (2024-01-02)
# All smart data models: https://smartdatamodels.org/extra/smartdatamodels.json 
# List of all subjects: https://github.com/smart-data-models/data-models/blob/master/specs/AllSubjects/official_list_data_models.json

#smart_data_model_context_urls = ['https://raw.githubusercontent.com/smart-data-models/dataModel.Battery/master/context.jsonld',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Building/master/context.jsonld',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Parking/master/context.jsonld',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.ParksAndGardens/master/context.jsonld',
#                         'https://uri.etsi.org/ngsi-ld/v1/ngsi-ld-core-context-v1.4.jsonld']

smart_data_model_urls = []
smart_data_models = []

####################################################################################################################
# ALL

#smart_data_model_urls = readSmartDataModelURLs()

#smart_data_model_context_urls = [ 'https://smart-data-models.github.io/data-models/context.jsonld' ]

####################################################################################################################
# Aeronautics

#smart_data_model_urls = [ 'https://raw.githubusercontent.com/smart-data-models/dataModel.Aeronautics/master/AircraftModel/schema.json',
#                          'https://raw.githubusercontent.com/smart-data-models/dataModel.Aeronautics/master/Aircraft/schema.json',
#                          'https://raw.githubusercontent.com/smart-data-models/dataModel.Aeronautics/master/Airline/schema.json',
#                          'https://raw.githubusercontent.com/smart-data-models/dataModel.Aeronautics/master/Airport/schema.json',
#                          'https://raw.githubusercontent.com/smart-data-models/dataModel.Aeronautics/master/FlightNotification/schema.json',
#                          'https://raw.githubusercontent.com/smart-data-models/dataModel.Aeronautics/master/Flight/schema.json' ]

#prefixes['aeronautics:']="<https://smartdatamodels.org/dataModel.Aeronautics/>"
#inversePrefixes["https://smartdatamodels.org/dataModel.Aeronautics/"] = "aeronautics:"

#smart_data_model_context_urls = [ 'https://smart-data-models.github.io/data-models/context.jsonld',
#                                  'https://raw.githubusercontent.com/smart-data-models/dataModel.Aeronautics/master/context.jsonld']


####################################################################################################################

# Agrifood
#smart_data_model_urls = ['https://raw.githubusercontent.com/smart-data-models/dataModel.Agrifood/master/AgriApp/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Agrifood/master/AgriCrop/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Agrifood/master/AgriFarm/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Agrifood/master/AgriGreenhouse/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Agrifood/master/AgriParcelOperation/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Agrifood/master/AgriParcelRecord/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Agrifood/master/AgriParcel/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Agrifood/master/AgriPest/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Agrifood/master/AgriProductType/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Agrifood/master/AgriSoil/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Agrifood/master/Animal/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Agrifood/master/Compartment/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Agrifood/master/Pen/schema.json']

#smart_data_model_urls = ['https://raw.githubusercontent.com/smart-data-models/dataModel.Agrifood/master/AgriFarm/schema.json']

#prefixes['agrifood:']="<https://smartdatamodels.org/dataModel.Agrifood/>"
#inversePrefixes["https://smartdatamodels.org/dataModel.Agrifood/"] = "agrifood:"

#smart_data_model_context_urls = [ 'https://smart-data-models.github.io/data-models/context.jsonld',
#                                  'https://raw.githubusercontent.com/smart-data-models/dataModel.Agrifood/e9c9bbd8a613a1850671a0e1184079a97a9d7190/context.jsonld']


####################################################################################################################

# Alert
# ALERT DOES NOT WORK AS ENTITY TYPE IS NOT SPECIFIED!!!
#smart_data_model_urls = ['https://raw.githubusercontent.com/smart-data-models/dataModel.Alert/master/Alert/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Alert/master/Anomaly/schema.json']

#prefixes['alert:']="<https://smartdatamodels.org/dataModel.Alert/>"
#inversePrefixes["https://smartdatamodels.org/dataModel.Alert/"] = "alert:"

#smart_data_model_context_urls = [ 'https://smart-data-models.github.io/data-models/context.jsonld',
#                                  'https://raw.githubusercontent.com/smart-data-models/dataModel.Alert/master/context.jsonld']

####################################################################################################################

# Aquaculture
#smart_data_model_urls = ['https://raw.githubusercontent.com/smart-data-models/dataModel.Aquaculture/master/BreedingOperation/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Aquaculture/master/FishContainment/schema.json']

#prefixes['aquaculture:']="<https://smartdatamodels.org/dataModel.Aquaculture/>"
#inversePrefixes["https://smartdatamodels.org/dataModel.Aquaculture/"] = "aquaculture:"

#smart_data_model_context_urls = [ 'https://smart-data-models.github.io/data-models/context.jsonld',
#                                  'https://raw.githubusercontent.com/smart-data-models/dataModel.Aquaculture/master/context.jsonld']
####################################################################################################################

# Battery
#smart_data_model_urls = ['https://raw.githubusercontent.com/smart-data-models/dataModel.Battery/master/BatteryStatus/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Battery/master/Battery/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Battery/master/StorageBatteryDevice/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Battery/master/StorageBatteryMeasurement/schema.json']

#prefixes['battery:']="<https://smartdatamodels.org/dataModel.Battery/>"
#inversePrefixes["https://smartdatamodels.org/dataModel.Battery/"] = "battery:"

#smart_data_model_context_urls = [ 'https://smart-data-models.github.io/data-models/context.jsonld',
#                                  'https://raw.githubusercontent.com/smart-data-models/dataModel.Battery/master/context.jsonld']

####################################################################################################################

# Building
#smart_data_model_urls = ['https://raw.githubusercontent.com/smart-data-models/dataModel.Building/master/BuildingOperation/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Building/master/Building/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Building/master/VibrationsObserved/schema.json' ]

#prefixes['building:']="<https://smartdatamodels.org/dataModel.Building/>"
#inversePrefixes["https://smartdatamodels.org/dataModel.Building/"] = "building:"

#smart_data_model_context_urls = [ 'https://smart-data-models.github.io/data-models/context.jsonld',
#                                  'https://raw.githubusercontent.com/smart-data-models/dataModel.Building/master/context.jsonld']

####################################################################################################################

# Device
#smart_data_model_urls = ['https://raw.githubusercontent.com/smart-data-models/dataModel.Device/master/DeviceModel/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Device/master/Device/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Device/master/PrivacyObject/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Device/master/SmartMeteringObservation/schema.json' ]

#prefixes['device:']="<https://smartdatamodels.org/dataModel.Device/>"
#inversePrefixes["https://smartdatamodels.org/dataModel.Device/"] = "device:"

#smart_data_model_context_urls = [ 'https://smart-data-models.github.io/data-models/context.jsonld',
#                                  'https://raw.githubusercontent.com/smart-data-models/dataModel.Device/master/context.jsonld']

####################################################################################################################
# Energy

#smart_data_model_urls = ['https://raw.githubusercontent.com/smart-data-models/dataModel.Energy/master/ACMeasurement/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Energy/master/InverterDevice/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Energy/master/TechnicalCabinetDevice/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Energy/master/ThreePhaseAcMeasurement/schema.json']#
#
#prefixes['energy:']="<https://smartdatamodels.org/dataModel.Energy/>"
#inversePrefixes["https://smartdatamodels.org/dataModel.Energy/"] = "energy:"
#
#smart_data_model_context_urls = [ 'https://smart-data-models.github.io/data-models/context.jsonld',
#                                  'https://raw.githubusercontent.com/smart-data-models/dataModel.Energy/master/context.jsonld']

####################################################################################################################

# Environment
#smart_data_model_urls = ['https://raw.githubusercontent.com/smart-data-models/dataModel.Environment/master/AeroAllergenObserved/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Environment/master/AirQualityObserved/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Environment/master/ElectroMagneticObserved/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Environment/master/FloodMonitoring/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Environment/master/IndoorEnvironmentObserved/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Environment/master/NoiseLevelObserved/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Environment/master/PhreaticObserved/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Environment/master/RainFallRadarObservation/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Environment/master/WaterObserved/schema.json']

#prefixes['environment:']="<https://smartdatamodels.org/dataModel.Environment/>"
#inversePrefixes["https://smartdatamodels.org/dataModel.Environment/"] = "environment:"

#smart_data_model_context_urls = [ 'https://smart-data-models.github.io/data-models/context.jsonld',
#                                  'https://raw.githubusercontent.com/smart-data-models/dataModel.Environment/master/context.jsonld']

####################################################################################################################

# Green Energy
#smart_data_model_urls = ['https://raw.githubusercontent.com/smart-data-models/dataModel.GreenEnergy/master/GreenEnergyGenerator/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.GreenEnergy/master/GreenEnergyMeasurement/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.GreenEnergy/master/PhotovoltaicDevice/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.GreenEnergy/master/PhotovoltaicMeasurement/schema.json']

#prefixes['greenenergy:']="<https://smartdatamodels.org/dataModel.GreenEnergy/>"
#inversePrefixes["https://smartdatamodels.org/dataModel.GreenEnergy/"] = "greenenergy:"

#smart_data_model_context_urls = [ 'https://smart-data-models.github.io/data-models/context.jsonld',
#                                  'https://raw.githubusercontent.com/smart-data-models/dataModel.GreenEnergy/master/context.jsonld']


####################################################################################################################

# Parks and Gardens
#smart_data_model_urls = ['https://raw.githubusercontent.com/smart-data-models/dataModel.ParksAndGardens/master/FlowerBed/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.ParksAndGardens/master/Garden/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.ParksAndGardens/master/GreenspaceRecord/schema.json']

#prefixes['parks:']="<https://smartdatamodels.org/dataModel.ParksAndGardens>"
#inversePrefixes["https://smartdatamodels.org/dataModel.ParksAndGardens/"] = "parks:"

#smart_data_model_context_urls = [ 'https://smart-data-models.github.io/data-models/context.jsonld',
#                                  'https://raw.githubusercontent.com/smart-data-models/dataModel.ParksAndGardens/master/context.jsonld']

####################################################################################################################

# Parking
#smart_data_model_urls = ['https://raw.githubusercontent.com/smart-data-models/dataModel.Parking/master/OffStreetParking/schema.json',                  
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Parking/master/OnStreetParking/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Parking/master/ParkingAccess/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Parking/master/ParkingGroup/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Parking/master/ParkingSpot/schema.json']

#prefixes['parking:']="<https://smartdatamodels.org/dataModel.Parking/>"
#inversePrefixes["https://smartdatamodels.org/dataModel.Parking/"] = "parking:"

#smart_data_model_context_urls = [ 'https://smart-data-models.github.io/data-models/context.jsonld',
#                                  'https://raw.githubusercontent.com/smart-data-models/dataModel.Parking/master/context.jsonld']

####################################################################################################################

# Streetlighting
#smart_data_model_urls = ['https://raw.githubusercontent.com/smart-data-models/dataModel.Streetlighting/master/StreetlightControlCabinet/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Streetlighting/master/StreetlightGroup/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Streetlighting/master/StreetlightModel/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Streetlighting/master/Streetlight/schema.json']
                         
#prefixes['streetlight:']="<https://smartdatamodels.org/dataModel.Streetlighting/>"
#inversePrefixes["https://smartdatamodels.org/dataModel.Streetlighting/"] = "streetlight:"
                         
                         
#smart_data_model_context_urls = [ 'https://smart-data-models.github.io/data-models/context.jsonld',
#                                  'https://raw.githubusercontent.com/smart-data-models/dataModel.Streetlighting/master/context.jsonld']

####################################################################################################################

# Transportation
#smart_data_model_urls = ['https://raw.githubusercontent.com/smart-data-models/dataModel.Transportation/master/BikeHireDockingStation/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Transportation/master/BikeLane/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Transportation/master/CityWork/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Transportation/master/CrowdFlowObserved/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Transportation/master/EVChargingStation/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Transportation/master/ItemFlowObserved/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Transportation/master/RestrictedTrafficArea/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Transportation/master/RestrictionException/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Transportation/master/RoadAccident/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Transportation/master/RoadSegment/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Transportation/master/Road/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Transportation/master/SpecialRestriction/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Transportation/master/TrafficFlowObserved/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Transportation/master/TransportStation/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Transportation/master/VehicleModel/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Transportation/master/Vehicle/schema.json' ]

#prefixes['transportation:']="<https://smartdatamodels.org/dataModel.Transportation/>"
#inversePrefixes["https://smartdatamodels.org/dataModel.Transportation/"] = "transportation:"

#smart_data_model_context_urls = [ 'https://smart-data-models.github.io/data-models/context.jsonld',
#                                  'https://raw.githubusercontent.com/smart-data-models/dataModel.Transportation/master/context.jsonld']

####################################################################################################################

# Urban Mobility
#smart_data_model_urls = ['https://raw.githubusercontent.com/smart-data-models/dataModel.UrbanMobility/master/ArrivalEstimation/schema.json',                         'https://raw.githubusercontent.com/smart-data-models/dataModel.UrbanMobility/master/GtfsAccessPoint/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.UrbanMobility/master/GtfsAgency/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.UrbanMobility/master/GtfsCalendarDateRule/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.UrbanMobility/master/GtfsCalendarRule/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.UrbanMobility/master/GtfsFrequency/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.UrbanMobility/master/GtfsRoute/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.UrbanMobility/master/GtfsService/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.UrbanMobility/master/GtfsShape/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.UrbanMobility/master/GtfsStation/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.UrbanMobility/master/GtfsStopTime/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.UrbanMobility/master/GtfsStop/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.UrbanMobility/master/GtfsTransferRule/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.UrbanMobility/master/GtfsTrip/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.UrbanMobility/master/PublicTransportRoute/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.UrbanMobility/master/PublicTransportStop/schema.json' ]

#prefixes['urbmobility:']="<https://smartdatamodels.org/dataModel.UrbanMobility/>"
#inversePrefixes["https://smartdatamodels.org/dataModel.UrbanMobility/"] = "urbmobility:"

#smart_data_model_context_urls = [ 'https://smart-data-models.github.io/data-models/context.jsonld',
#                                  'https://raw.githubusercontent.com/smart-data-models/dataModel.UrbanMobility/master/context.jsonld']


####################################################################################################################

# Water Distribution Network
#smart_data_model_urls = ['https://raw.githubusercontent.com/smart-data-models/dataModel.WaterDistributionManagementEPANET/master/Curve/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.WaterDistributionManagementEPANET/master/Junction/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.WaterDistributionManagementEPANET/master/Network/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.WaterDistributionManagementEPANET/master/Pattern/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.WaterDistributionManagementEPANET/master/Pipe/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.WaterDistributionManagementEPANET/master/Pump/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.WaterDistributionManagementEPANET/master/Reservoir/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.WaterDistributionManagementEPANET/master/SimulationResult/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.WaterDistributionManagementEPANET/master/SimulationScenario/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.WaterDistributionManagementEPANET/master/Tank/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.WaterDistributionManagementEPANET/master/Valve/schema.json']

#prefixes['waterdist:']="<https://smartdatamodels.org/dataModel.WaterDistributionManagementEPANET/>"
#inversePrefixes["https://smartdatamodels.org/dataModel.WaterDistributionManagementEPANET/"] = "waterdist:"

#smart_data_model_context_urls = [ 'https://smart-data-models.github.io/data-models/context.jsonld',
#                                  'https://raw.githubusercontent.com/smart-data-models/dataModel.WaterDistributionManagementEPANET/master/context.jsonld']

####################################################################################################################

# WifiNetwork
#smart_data_model_urls = ['https://raw.githubusercontent.com/smart-data-models/dataModel.WifiNetwork/master/AccessPoint/schema.json',
#                         'https://raw.githubusercontent.com/smart-data-models/dataModel.WifiNetwork/master/WifiPointOfInterest/schema.json']

#prefixes['wifinetwork:']="<https://smartdatamodels.org/dataModel.WifiNetwork/>"
#inversePrefixes["https://smartdatamodels.org/dataModel.WifiNetwork/"] = "wifinetwork:"

#smart_data_model_context_urls = [ 'https://smart-data-models.github.io/data-models/context.jsonld',
#                                  'https://raw.githubusercontent.com/smart-data-models/dataModel.WifiNetwork/master/context.jsonld']

####################################################################################################################

# Building and Energy
smart_data_model_urls = ['https://raw.githubusercontent.com/smart-data-models/dataModel.Building/master/BuildingOperation/schema.json',
                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Building/master/Building/schema.json',
                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Building/master/VibrationsObserved/schema.json',
                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Energy/master/ACMeasurement/schema.json',
                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Energy/master/InverterDevice/schema.json',
                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Energy/master/TechnicalCabinetDevice/schema.json',
                         'https://raw.githubusercontent.com/smart-data-models/dataModel.Energy/master/ThreePhaseAcMeasurement/schema.json']


prefixes['building:']="<https://smartdatamodels.org/dataModel.Building/>"
inversePrefixes["https://smartdatamodels.org/dataModel.Building/"] = "building:"
prefixes['energy:']="<https://smartdatamodels.org/dataModel.Energy/>"
inversePrefixes["https://smartdatamodels.org/dataModel.Energy/"] = "energy:"

smart_data_model_context_urls = [ 'https://smart-data-models.github.io/data-models/context.jsonld',
                                  'https://raw.githubusercontent.com/smart-data-models/dataModel.Building/master/context.jsonld',
                                  'https://raw.githubusercontent.com/smart-data-models/dataModel.Energy/master/context.jsonld']


core_context.update(readJSONLDContext(core_context_url))

for context_url in smart_data_model_context_urls:
    context.update(readJSONLDContext(context_url))

    
for url in smart_data_model_urls:
    smart_data_models.append(readFromURL(url))

for data in smart_data_models:
    readSmartDataModel(data)


print("---------------------------------------------------------------")    
printOntology(filename="BuildingAndEnergy.ttl")
#printOntology(filename="All-WithoutEnergyCIM.ttl")


---------------------------------------------------------------
Printing to file: BuildingAndEnergy.ttl
Printed to file: BuildingAndEnergy.ttl


In [31]:
objectProperties

{'https://uri.etsi.org/ngsi-ld/GeoProperty': {'rdf:type': 'owl:ObjectProperty',
  'ngsild_type': 'https://uri.etsi.org/ngsi-ld/Property',
  'rdfs:range': [':Geometry']},
 'https://uri.etsi.org/ngsi-ld/LanguageProperty': {'rdf:type': 'owl:ObjectProperty',
  'ngsild_type': 'https://uri.etsi.org/ngsi-ld/Property',
  'rdfs:range': [':LanguageMap']},
 'https://uri.etsi.org/ngsi-ld/Property': {'rdf:type': 'owl:ObjectProperty',
  'rdfs:domain': ['https://uri.etsi.org/ngsi-ld/Entity',
   'https://uri.etsi.org/ngsi-ld/Property',
   'https://uri.etsi.org/ngsi-ld/Relationship'],
  'rdfs:range': ['https://uri.etsi.org/ngsi-ld/Property',
   'https://uri.etsi.org/ngsi-ld/Value']},
 'https://uri.etsi.org/ngsi-ld/Relationship': {'rdf:type': 'owl:ObjectProperty',
  'rdfs:domain': ['https://uri.etsi.org/ngsi-ld/Entity',
   'https://uri.etsi.org/ngsi-ld/Property',
   'https://uri.etsi.org/ngsi-ld/Relationship'],
  'rdfs:range': ['https://uri.etsi.org/ngsi-ld/Entity',
   'https://uri.etsi.org/ngsi-ld/Rela

In [30]:
datatypeProperties

{'https://uri.etsi.org/ngsi-ld/TemporalProperty': {'rdf:type': 'owl:DatatypeProperty'},
 'https://uri.etsi.org/ngsi-ld/createdAt': {'rdf:type': 'owl:DatatypeProperty',
  'rdfs:subPropertyOf': ':TemporalProperty',
  'rdfs:range': ['xsd:dateTime']},
 'https://uri.etsi.org/ngsi-ld/hasValue': {'rdf:type': 'owl:DatatypeProperty',
  'rdfs:domain': [':Property']},
 'https://uri.etsi.org/ngsi-ld/hasLanguageMap': {'rdf:type': 'owl:DatatypeProperty',
  'rdfs:subPropertyOf': ':hasValue',
  'rdfs:domain': [':LanguageProperty'],
  'rdfs:range': [':LanguageMap']},
 'https://uri.etsi.org/ngsi-ld/modifiedAt': {'rdf:type': 'owl:DatatypeProperty',
  'rdfs:subPropertyOf': ':TemporalProperty',
  'rdfs:range': ['xsd:dateTime']},
 'https://uri.etsi.org/ngsi-ld/observedAt': {'rdf:type': 'owl:DatatypeProperty',
  'rdfs:subPropertyOf': ':TemporalProperty',
  'rdfs:range': ['xsd:dateTime']},
 'https://uri.etsi.org/ngsi-ld/unitCode': {'rdf:type': 'owl:DatatypeProperty',
  'rdfs:range': ['xsd:string']},
 'https:/

In [31]:
definitions

{'https://smart-data-models.github.io/data-models/terms.jsonld#/definitions/email': {'owl:onDatatype': 'xsd:string',
  'rdfs:comment': 'idn-email. Property. Email address of owner.'},
 'https://smart-data-models.github.io/data-models/common-schema.json#/definitions/userAlias': {'owl:onDatatype': 'xsd:string',
  'rdfs:comment': 'Property. An anonymous alias of a user.'},
 'https://smart-data-models.github.io/data-models/terms.jsonld#/definitions/tag': {'owl:onDatatype': 'xsd:string',
  'rdfs:comment': "Property. Model:'https://schema.org/Text'. An optional text string used to qualify an item"},
 'https://smart-data-models.github.io/data-models/common-schema.json#/definitions/timeInstant': {'owl:onDatatype': 'xsd:dateTime',
  'rdfs:comment': "Property. Model:'https://schema.org/Datetime'. Timestamp of the payload . There can be production environments where the attribute type is equal to the `ISO8601` string. If so, it must be considered as a synonym of `DateTime`. This attribute is kept